Decision trees approaches.

CatBoost:

1). Can process with a lot of different kind of features out of box

2). Accurately learning based on previous trees results

3). Has own techniques to work with missing values out of box

Random Forest:

1). Fastly, independently and stability learn with subsets of features and data with subsets prallel trees training (bagging)

Both:

1). No need feature selection, so all data can be useful

Final models for each method serialized in pickle files.

Output:
    
    CatBoost:
    Accuracy: 0.9082952215738842
    Precision: 0.6593406593406593
    Recall: 0.09828009828009827
    ROC-AUC 0.8490243310164115
    
    Random Forest:
    Accuracy: 0.9040372181044
    Precision: 0.7
    Recall: 0.005733005733005733
    ROC-AUC 0.7684109230056015

In [1]:
#!pip install scikit-learn==1.2.2 --user --force
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score
from scipy.stats import randint
import pickle
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv(r'../dataset.csv',
                 header=None, encoding='utf8', delimiter=',', index_col=None)
df = df.rename(columns={0:'id', 1:'sample_date', 2:'target'})

string_col = []
for i in df.columns:
    if is_numeric_dtype(df[i]):
        try:
            mean_1 = df[i].mean().values[0]
            df[i].fillna(value=mean_1, inplace=True)
        except:
            df[i].fillna(value=0, inplace=True)
    if is_string_dtype(df[i]):
        string_col.append(i)
        try:
            mode_2 = df[i].mode().values[0]
            df[i].fillna(value=mode_2, inplace=True)
        except:
            df[i].fillna(value='0', inplace=True)

df = pd.get_dummies(df, columns=string_col, drop_first=True)
df.columns = df.columns.astype(str)# Обучающая и тестовая выборки


X = df.drop('target', axis=1)
y = df[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

features = [col_name for col_name in df.columns if col_name != 'make']
cat_features = [col_name for col_name in features if df[col_name].dtype == 'object']

# CatBoost

In [8]:
#############Create pickle model
# param_dist_cb = {
#             'iterations': [500, 1000],
#             'depth': [10, 20],
#             'verbose': [False],
#             }

# cb_clf = CatBoostClassifier()

# grid_search_cb = GridSearchCV(cb_clf, param_dist_cb, cv=5)

# cb_clf.fit(X_train, y_train, 
#         cat_features=cat_features, 
#         eval_set=(X_test, y_test), 
#         verbose=False)

# pickle.dump(cb_clf, open("./catboost.pickle", "wb"))
#############Load from pickle model
with open('./catboost.pickle', 'rb') as f:
    catboost_estimator = pickle.load(f)

y_pred = catboost_estimator.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Confusion matrix", cm)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC-AUC", roc_auc_score(y, catboost_estimator.predict_proba(X)[:, 1]))

Confusion matrix [[11399    62]
 [ 1101   120]]
Accuracy: 0.9082952215738842
Precision: 0.6593406593406593
Recall: 0.09828009828009827
ROC-AUC 0.8490243310164115


# Random Forest

In [9]:
#############Create pickle model
# param_dist = {
#             'n_estimators': randint(200, 500),
#             'max_depth': randint(8, 10)
#         }

# rf_clf = RandomForestClassifier()

# rand_search = RandomizedSearchCV(rf_clf,
#                                  param_distributions = param_dist,
#                                  n_iter=2,
#                                  cv=5)

# rand_search.fit(X_train, y_train.values.ravel())

# best_rf = rand_search.best_estimator_

# print('Best hyperparameters:',  rand_search.best_params_)

# pickle.dump(best_rf, open("./random_forest.pickle", "wb"))
#############Load from pickle model
with open('./random_forest.pickle', 'rb') as f:
    random_forest_estimator = pickle.load(f)

y_pred = random_forest_estimator.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Confusion matrix", cm)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC-AUC", roc_auc_score(y, random_forest_estimator.predict_proba(X)[:, 1]))

Confusion matrix [[11458     3]
 [ 1214     7]]
Accuracy: 0.9040372181044
Precision: 0.7
Recall: 0.005733005733005733
ROC-AUC 0.7684109230056015
